In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from timm.models.efficientnet import efficientnetv2_m
#import tensorflow.keras.applications.efficientnet as efn
!pip install timm
import timm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import time


from pathlib import Path
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
 
import os
for dirname, _, filenames in os.walk('/kaggle/input/vexcompetitionexpanded'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-04-21 21:59:00.199460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 21:59:00.199588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 21:59:00.489452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/vexcompetitionexpanded/notes.json
/kaggle/input/vexcompetitionexpanded/classes.txt
/kaggle/input/vexcompetitionexpanded/labels/f45f738c-data1.txt
/kaggle/input/vexcompetitionexpanded/labels/98c6b011-data109.txt
/kaggle/input/vexcompetitionexpanded/labels/b7daf22a-data118.txt
/kaggle/input/vexcompetitionexpanded/labels/f0e10d90-data20.txt
/kaggle/input/vexcompetitionexpanded/labels/210c603a-data138.txt
/kaggle/input/vexcompetitionexpanded/labels/f19d8fd0-data51.txt
/kaggle/input/vexcompetitionexpanded/labels/7278e187-data111.txt
/kaggle/input/vexcompetitionexpanded/labels/c6a53bb1-data105.txt
/kaggle/input/vexcompetitionexpanded/labels/b7348070-data142.txt
/kaggle/input/vexcompetitionexpanded/labels/37575e82-data32.txt
/kaggle/input/vexcompetitionexpanded/labels/00ae3865-data133.txt
/kaggle/input/vexcompetitionexpanded/labels/6bf453c8-data4.txt
/kaggle/input/vexcompetitionexpanded/labels/5289065a-data147.txt
/kaggle/input/vexcompetitionexpanded/labels/1524c45b-data14.txt
/

In [2]:
all_devices = tf.config.list_physical_devices()
all_devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
if 'GPU' in str(all_devices):
    # Use GPUs for training/inference
    devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.list_physical_devices()
    device_names = [d.name.split('e:')[1] for d in devices]
    # Multiple GPUS
    #strategy = tf.distribute.MirroredStrategy(devices=device_names[:len(devices)])
    # One GPU
    strategy = tf.distribute.OneDeviceStrategy(device_names[0])
    print(f'Using GPU for training...')
    device_name_for_file = f'GPU_x{len(device_names)}'
elif 'TPU' in str(all_devices):
    # Use TPU for training
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print(f'Using TPUs for training...')
    device_name_for_file = 'TPU'
else:
    # Run on CPU
    devices = tf.config.experimental.list_physical_devices('CPU')
    device_names = [d.name.split('e:')[1] for d in devices]
    strategy = tf.distribute.OneDeviceStrategy(device_names[0])
    print(f'Using CPUs for training...')
    device_name_for_file = 'CPU'



Using GPU for training...


In [4]:
# If mnist_train.csv and mnist_test.csv are not /kaggle/input,
# then specify the correct directory to where they are located
!ls /kaggle/input/assignment-3-spring2024
dataset_location = Path('/kaggle/input/Yolo-labels-4_12_24')

sampleSubmission.csv  test_images.npy  train_images.npy  train_labels.npy


In [5]:
train_images_dir = '/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images'
train_labels_dir = '/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/labels'

In [6]:
'''
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=2,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.01, # Randomly zoom image 10%
        width_shift_range=0.01,  # randomly shift images horizontally 10%
        height_shift_range=0.01,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(train_images_dir)
'''

'\ndatagen = ImageDataGenerator(\n        featurewise_center=False,  # set input mean to 0 over the dataset\n        samplewise_center=False,  # set each sample mean to 0\n        featurewise_std_normalization=False,  # divide inputs by std of the dataset\n        samplewise_std_normalization=False,  # divide each input by its std\n        zca_whitening=False,  # dimesion reduction\n        rotation_range=2,  # randomly rotate images in the range 5 degrees\n        zoom_range = 0.01, # Randomly zoom image 10%\n        width_shift_range=0.01,  # randomly shift images horizontally 10%\n        height_shift_range=0.01,  # randomly shift images vertically 10%\n        horizontal_flip=False,  # randomly flip images\n        vertical_flip=False)  # randomly flip images\n\ndatagen.fit(train_images_dir)\n'

In [7]:
import cv2
#private array of image and annotation paths
image_paths = []
label_paths = []

# Iterate over the image directory
for img_path in os.listdir(train_images_dir):
    image_path = os.path.join(train_images_dir, img_path)
    print(image_path)
    #form annotation path from annotation directory then the first part of the image name without extension, then add .txt extension
    annotation_path = os.path.join(train_labels_dir, os.path.splitext(img_path)[0] + '.txt')
    #print(annotation_path)
    #if os.path.isfile(image_path) and os.path.isfile(annotation_path):#if the files exist at both places
    image_paths.append(image_path)
    label_paths.append(annotation_path)
        #add them to list
#index in index array


np.array(image_paths).shape
np.array(label_paths).shape

/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/ad6c2e92-data124.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/a65879d7-data8.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/d0658f5c-data22.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/b7daf22a-data118.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/b97963d7-data132.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/f5a2b7fe-data136.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/37575e82-data32.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/f19d8fd0-data51.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/f75582ad-data25.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/46666ce7-data121.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/b7348070-data142.jpg
/kaggle/input/vexcompetitionexpanded/Yolo-labels-4_12_24/images/0707d59

(116,)

In [8]:
images = []


#for every image in batch get the image
for path in image_paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)

train_images = np.array(images)
train_images.shape

(116, 480, 640, 3)

In [9]:
bboxes = []
for path in label_paths:
    #print(path)
    #open annotation path, turn into a bounding box array of floats
    with open(path, 'r') as f:
        lines = f.readlines()
        if(lines == []):
          #THis is if there is not an object
          #this solution sucks but stops crashes
          #ideal is to hhave the bot move straight in this case, or maybe turn
          #not sure what to put here
          bbox = [0,0,0,0,0]
          bboxes.append(bbox)
        else:
          bbox = [float(x) for x in lines[0].split(" ")]
          bboxes.append(bbox)
#convert to numpy array and return
train_labels = np.array(bboxes)

In [10]:
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size = 0.2, random_state = 42
)

In [11]:
'''
class BoundingBoxDatasetPaths(tf.keras.utils.Sequence):
    def __init__(self, image_paths, annotation_paths, batch_size=4, shuffle=True):
        self.image_paths = image_paths
        self.annotation_paths = annotation_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_paths))

    def __len__(self):
        return len(self.image_paths) // self.batch_size // 16

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = []
        bboxes = []

        for i in batch_indexes:
            img = cv2.imread(self.image_paths[i])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)

            with open(self.annotation_paths[i], 'r') as f:
                lines = f.readlines()
                bbox = [float(x) for x in lines[0].split()]
                bboxes.append(bbox)

        images = np.array(images)
        bboxes = np.array(bboxes)
        
       
            
        return images, bboxes
        '''

"\nclass BoundingBoxDatasetPaths(tf.keras.utils.Sequence):\n    def __init__(self, image_paths, annotation_paths, batch_size=4, shuffle=True):\n        self.image_paths = image_paths\n        self.annotation_paths = annotation_paths\n        self.batch_size = batch_size\n        self.shuffle = shuffle\n        self.indexes = np.arange(len(self.image_paths))\n\n    def __len__(self):\n        return len(self.image_paths) // self.batch_size // 16\n\n    def __getitem__(self, idx):\n        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]\n\n        images = []\n        bboxes = []\n\n        for i in batch_indexes:\n            img = cv2.imread(self.image_paths[i])\n            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n            images.append(img)\n\n            with open(self.annotation_paths[i], 'r') as f:\n                lines = f.readlines()\n                bbox = [float(x) for x in lines[0].split()]\n                bboxes.append(bbox)\n\n       

In [12]:
!pip install -U torch torchvision
!pip install yolov5 
import os
import cv2
import numpy as np

class BoundingBoxDataset(tf.keras.utils.Sequence):
    def __init__(self, image_dir, annotation_dir, batch_size=4, shuffle=True):
        #print("FEOFAF")
        self.image_paths = []
        self.annotation_paths = []
        self.batch_size = batch_size
        self.shuffle = shuffle
        #print(os.listdir(image_dir))
        #print(os.listdir(annotation_dir))
        # Iterate over the image directory
        for img_path in os.listdir(image_dir):
            image_path = os.path.join(image_dir, img_path)
            annotation_path = os.path.join(annotation_dir, os.path.splitext(img_path)[0] + '.txt')

            if os.path.isfile(image_path) and os.path.isfile(annotation_path):
                self.image_paths.append(image_path)
                self.annotation_paths.append(annotation_path)

        self.indexes = np.arange(len(self.image_paths))
        #print(len(self.image_paths),"EEHE")
        #print(self.image_paths[5])

    def __len__(self):
       # print("ive been called")
        #print(len(self.image_paths),"current image path length")
        #print(self.batch_size,"current batch size")
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        #print("HELP",idx)
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = []
        bboxes = []

        for i in batch_indexes:
            img = cv2.imread(self.image_paths[i])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)

            with open(self.annotation_paths[i], 'r') as f:
                lines = f.readlines()
                if lines:
                    bbox = [float(x) for x in lines[0].split()]
                    bbox = bbox[1:]
                    bboxes.append(bbox)
                else:
                    print(f"Warning: Empty annotation file for image {self.image_paths[i]}")
                    bbox = [0,0,0,0]
                    bboxes.append(bbox)
                    continue
        print(f"Batch {idx}: Number of images = {len(images)}, Number of bboxes = {len(bboxes)}")

        if len(images) != self.batch_size or len(bboxes) != self.batch_size:
            print("Warning: Inconsistent batch size")
        images = np.array(images)
        bboxes = np.array(bboxes)

        return images, bboxes

 
 


#Defining the model architeture using basic convolution
    #inputs
inputs = tf.keras.Input(shape=(480,640,3))
#tanh activation function
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (16, 16), padding='same', input_shape=(480, 640, 3))) #grab last layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(32, (16, 16), padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(8,8))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(64, (8, 8),padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(64, (8, 8), padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(4,4))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(128, (3, 3),  padding='same',))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(units=2, activation='softmax'))
#model = tf.keras.Model(inputs=inputs, outputs=outputs)
#maybe modify dropout and amount of batch normalization we do
#model summary
#model.summary()

#model = Model(inputs=inputs, outputs=outputs)

#Freeze layers aka done training
#for layer in backbone.layers:
   # layer.trainable = False
#!pip install tensorflow-addons
#from tensorflow_addons.losses import huber_loss
model.compile(optimizer='adam', loss=MeanSquaredError())

dataset = BoundingBoxDataset(train_images_dir, train_labels_dir)

print(dataset)
num_epochs = 15
for epoch in range(num_epochs):
    #images,bboxes = dataset.__getitem__()
    #print(len(dataset),"dsgajhgjsd")
    for images, bboxes in dataset:
  
        loss = model.train_on_batch(images, bboxes)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')

model.save('new_triball_tracker.keras')
# serialize model to JSON
model_json = model.to_json()
with open("triball_tracker.json", "w") as json_file:
     json_file.write(model_json) 
 
model.save_weights("triball_tracker.weights.h5")
print("Saved model to disk") 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 9.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 4.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 8.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Batch 0: Number of images = 4, Number of bboxes = 4


ValueError: Dimensions must be equal, but are 4 and 2 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](compile_loss/mean_squared_error/Cast, sequential_1/dense_1_2/Softmax)' with input shapes: [4,4], [4,2].

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=10,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(train_images)

for img_path in os.listdir(train_images_dir):  # Assume you have a list of image paths
    img = load_img(img_path)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir='path_to_save', save_prefix='aug', save_format='jpeg'):
        i += 1
        if i > 52:
            break

In [ ]:
import os
import cv2
image_dir = '/kaggle/input/unlabeledtriballs/data_old'

for filename in os.listdir(image_dir):
    #Sanity check
     if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_dir, filename)
        
        # Generic preprocessing
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        input_img = np.expand_dims(img, axis=0)
        
        predictions = model.predict(input_img)


        for bbox in predictions[0]:
            if type(bbox.item()) == type(0.0):
                print("skip ",bbox)
                continue
            
            x1, y1, x2, y2, _ = bbox
            print(x1)
            print(y1)
            print(x2)
            print(y2)
            print(bbox)
            #x1 = int(x1 * img.shape[1])
            #y1 = int(y1 * img.shape[0])
            #x2 = int(x2 * img.shape[1])
           # y2 = int(y2 * img.shape[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        #testing code frankly
       # cv2.imshow('Image with Bounding Boxes', img)
       # cv2.waitKey(0)
       # cv2.destroyAllWindows()

In [ ]:

#model.compile(optimizer='adam',
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          #    metrics=['accuracy'])
#3
#history = model.fit(train_images, train_labels, epochs=10,
               #     validation_data=(test_images, test_labels))


In [ ]:
file_datetime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%S')
filepath = f'./model_{file_datetime}_{device_name_for_file}.keras'
print(f'Best model will be saved to: {Path.cwd() / filepath}')

In [ ]:
# Train the model, using datagen data

'''
BATCH_SIZE = 4
NUM_EPOCHS = 30
#opt_rms = tf.keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
MIN_VAL_LOSS = 0#np.inf#bad name, trying now to optimize accuracy
loss_history = [[],[]]
acc_history = [[],[]]
total_train_time = 0.0
model.compile(loss='categorical_crossentropy',
        optimizer='adam', #maybe change optimizer?
        metrics=['accuracy'])
kf = KFold(n_splits=10, shuffle=True, random_state=42)#maybe use? maybe change to 5?
# Training loop
# NOTE: This works better than callbacks
for i in range(0, NUM_EPOCHS):
    print(f'EPOCH: {i+1}/{NUM_EPOCHS}')
    start_epoch = time.time()

    history = model.fit(
        train_images,train_labels,
        #datagen.flow(train_images,train_labels_cat, batch_size=BATCH_SIZE),
        validation_data=(val_images, val_labels),
        epochs=1,
        verbose = 1
    )
    end_epoch = time.time()#https://storage.googleapis.com/kaggle-avatars/thumbnails/default-thumb.png
    delta_epoch = end_epoch - start_epoch
    total_train_time += delta_epoch
    print(f'LOG --> This epoch took {delta_epoch}...')
    if history.history['val_accuracy'][0] > MIN_VAL_LOSS: #val_accuracy remember to switch the comparison when switching
        print(f'LOG --> val_accuracy improved from {MIN_VAL_LOSS} to {history.history["val_accuracy"][0]}...')
        print(f'LOG --> saving model as {filepath}')
        MIN_VAL_LOSS = history.history['val_accuracy'][0]
        model.save(filepath, overwrite=True)
    else:
        print(f'LOG --> val_accuracy did not improve...')
    # Keep track of the training history
    loss_history[0].append(history.history['loss'][0])
    loss_history[1].append(history.history['val_loss'][0])
    acc_history[0].append(history.history['accuracy'][0])
    acc_history[1].append(history.history['val_accuracy'][0])
print(f'LOG--> Total training time: {total_train_time}')

In [ ]:
'''plt.plot(acc_history[0], color='black',)
plt.plot(acc_history[1], color='red',)
plt.title('Training Accuracy vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

plt.plot(loss_history[0])
plt.plot(loss_history[1])
plt.title('Training Loss vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
'''import csv

# Load the best model
print(f'Loading saved model from {filepath}...')
best_model = tf.keras.models.load_model(filepath)

classes = {
    0: 'airplane',
    1: 'car',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

# Warm up device
print('Starting Warmup...')
with open('submission.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    count = 1
    writer.writerow(['id','label'])
    for prediction in best_model.predict(test_images, batch_size=BATCH_SIZE, verbose=0):
        value = np.argmax(prediction)
        writer.writerow([count,classes[value]])
        count+=1

